# Common imports & library functions

In [3]:
import collections
from collections import defaultdict, Counter
from dataclasses import dataclass
import doctest
import functools
import itertools
from itertools import count
import math
import re
from copy import deepcopy

# Day 11: Seating System

In [7]:
def _floors(n):
    return ['.'] * n

class SeatLayout:
    def __init__(self, initial_state, neighbor_threshold=4):
        self._neighbor_threshold = neighbor_threshold
        initial_state = initial_state.strip().replace(' ', '')
        cells = [list(row.strip()) for row in initial_state.split()]
        w = len(cells[0])
        self._cells = ([_floors(w + 2)] +
                       [_floors(1) + row + _floors(1) for row in cells] +
                       [_floors(w + 2)])

    def at(self, x, y):
        return self._cells[y+1][x+1]

    def set(self, x, y, state):
        self._cells[y+1][x+1] = state

    def cells(self):
        for y in range(1, len(self._cells) - 1):
            for x in range(1, len(self._cells[0]) - 1):
                yield (x - 1, y - 1, self._cells[y][x])

    def neighbors(self, x, y):
        for dx in (-1, 0, 1):
            for dy in (-1, 0, 1):
                if dx == 0 and dy == 0: continue
                yield self.at(x+dx, y+dy)

    def update(self):
        prev_layout = deepcopy(self)
        updated = False
        for x, y, cell in prev_layout.cells():
            neighbors = list(prev_layout.neighbors(x, y))
            if (cell == 'L' and not any(n == '#' for n in neighbors)):
                self.set(x, y, '#')
                updated = True
            elif (cell == '#' and neighbors.count('#') >= self._neighbor_threshold):
                self.set(x, y, 'L')
                updated = True
        return updated

    def num_occupied(self):
        return sum(r.count('#') for r in self._cells)

    def __str__(self):
        return '\n'.join(''.join(r) for r in self._cells)

class SeatLayout2(SeatLayout):
    def __init__(self, initial_state, neighbor_threshold=5):
        super().__init__(initial_state, neighbor_threshold)

    def within_grid(self, x, y):
        return 0 <= y < len(self._cells) - 1 and 0 <= x < len(self._cells[0]) - 1

    def neighbors(self, x, y):
        for dx in (-1, 0, 1):
            for dy in (-1, 0, 1):
                if dx == 0 and dy == 0: continue
                for xp, yp in zip(count(x+dx, dx), count(y+dy, dy)):
                    if not self.within_grid(xp, yp):
                        break
                    if (cell := self.at(xp, yp)) != '.':
                        yield cell
                        break

def simulate_until_equilibrium(layout):
    while layout.update():
        continue
    return layout.num_occupied()

In [8]:
test_layout = """
L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL
"""

assert simulate_until_equilibrium(SeatLayout(test_layout)) == 37
assert simulate_until_equilibrium(SeatLayout2(test_layout)) == 26

In [57]:
# Final answers
with open('day11.txt') as f:
    initial_state = f.read()
    print('Part 1: ', simulate_until_equilibrium(SeatLayout(initial_state)))
    print('Part 2: ', simulate_until_equilibrium(SeatLayout2(initial_state)))

Part 1:  2418
Part 2:  2144


# Day 12: Rain Risk

In [6]:
import math

def move_along_angle(x, y, distance, heading):
    """
    >>> move_along_angle(0, 0, 10, 90)
    (0, 10)
    >>> move_along_angle(4, 1 + 3 * math.sqrt(3), 6, 240)
    (1, 1)
    """
    angle = math.radians(heading)
    x += distance * math.cos(angle)
    y += distance * math.sin(angle)
    return int(round(x)), int(round(y))

def parse_actions(actions):
    for action in actions.strip().split():
        action = action.strip()
        yield action[0], int(action[1:])

def move(actions, start_pos=(0, 0)):
    """
    >>> move('''
    ...     F10
    ...     N3
    ...     F7
    ...     R90
    ...     F11
    ... ''')
    (17, -8)
    """
    x, y = start_pos
    heading = 0  # degrees
    for action, amount in parse_actions(actions):
        if action == 'N':
            y += amount
        elif action == 'S':
            y -= amount
        elif action == 'E':
            x += amount
        elif action == 'W':
            x -= amount
        elif action == 'L':
            heading = (heading + amount) % 360
        elif action == 'R':
            heading = (heading - amount) % 360
        elif action == 'F':
            x, y = move_along_angle(x, y, amount, heading)
    return x, y

In [7]:
doctest.run_docstring_examples(move_along_angle, globs=None, verbose=True)
doctest.run_docstring_examples(move, globs=None, verbose=True)

Finding tests in NoName
Trying:
    move_along_angle(0, 0, 10, 90)
Expecting:
    (0, 10)
ok
Trying:
    move_along_angle(4, 1 + 3 * math.sqrt(3), 6, 240)
Expecting:
    (1, 1)
ok
Finding tests in NoName
Trying:
    move('''
        F10
        N3
        F7
        R90
        F11
    ''')
Expecting:
    (17, -8)
ok


In [8]:
def rotate(x, y, angle):
    """
    >>> rotate(10, 4, -90)
    (4, -10)
    """
    angle = math.radians(angle)
    xp = x * math.cos(angle) - y * math.sin(angle)
    yp = x * math.sin(angle) + y * math.cos(angle)
    return int(round(xp)), int(round(yp))

def move_waypoint(actions, start_pos=(10, 1)):
    """
    >>> move_waypoint('''
    ...     F10
    ...     N3
    ...     F7
    ...     R90
    ...     F11
    ... ''')
    (214, -72)
    """
    sx, sy = 0, 0
    wx, wy = start_pos
    heading = 0  # degrees
    for action, amount in parse_actions(actions):
        if action == 'N':
            wy += amount
        elif action == 'S':
            wy -= amount
        elif action == 'E':
            wx += amount
        elif action == 'W':
            wx -= amount
        elif action == 'L':
            wx, wy = rotate(wx, wy, amount)
            heading = (heading + amount) % 360
        elif action == 'R':
            wx, wy = rotate(wx, wy, -amount)
            heading = (heading - amount) % 360
        elif action == 'F':
            sx += wx * amount
            sy += wy * amount
    return sx, sy

In [9]:
doctest.run_docstring_examples(rotate, globs=None, verbose=True)
doctest.run_docstring_examples(move_waypoint, globs=None, verbose=True)

Finding tests in NoName
Trying:
    rotate(10, 4, -90)
Expecting:
    (4, -10)
ok
Finding tests in NoName
Trying:
    move_waypoint('''
        F10
        N3
        F7
        R90
        F11
    ''')
Expecting:
    (214, -72)
ok


In [10]:
# Final answers
with open('day12.txt') as f:
    actions = f.read()
    print('Part 1: ', sum(abs(c) for c in move(actions)))
    print('Part 2: ', sum(abs(c) for c in move_waypoint(actions)))

Part 1:  1106
Part 2:  107281


In [92]:
move('''F10
N3
F7
R90
F11''')

0 0 0 F 10 -> 10 0 0
10 0 0 N 3 -> 10 3 0
10 3 0 F 7 -> 17 3 0
17 3 0 R 90 -> 17 3 270
17 3 270 F 11 -> 17 -8 270


(17, -8)

In [94]:
move('\n'.join(actions.split()[:100]))

0 0 0 F 77 -> 77 0 0
77 0 0 E 4 -> 81 0 0
81 0 0 S 2 -> 81 -2 0
81 -2 0 W 1 -> 80 -2 0
80 -2 0 L 180 -> 80 -2 90
80 -2 90 N 4 -> 80 2 90
80 2 90 R 180 -> 80 2 0
80 2 0 S 3 -> 80 -1 0
80 -1 0 W 5 -> 75 -1 0
75 -1 0 F 86 -> 161 -1 0
161 -1 0 L 90 -> 161 -1 90
161 -1 90 E 1 -> 162 -1 90
162 -1 90 F 16 -> 162 15 90
162 15 90 R 90 -> 162 15 0
162 15 0 N 1 -> 162 16 0
162 16 0 E 1 -> 163 16 0
163 16 0 F 86 -> 249 16 0
249 16 0 S 1 -> 249 15 0
249 15 0 F 36 -> 285 15 0
285 15 0 E 2 -> 287 15 0
287 15 0 L 180 -> 287 15 90
287 15 90 N 5 -> 287 20 90
287 20 90 F 46 -> 287 66 90
287 66 90 N 1 -> 287 67 90
287 67 90 L 90 -> 287 67 180
287 67 180 F 43 -> 244 67 180
244 67 180 S 5 -> 244 62 180
244 62 180 R 90 -> 244 62 90
244 62 90 F 41 -> 244 103 90
244 103 90 W 5 -> 239 103 90
239 103 90 N 1 -> 239 104 90
239 104 90 F 65 -> 239 169 90
239 169 90 E 4 -> 243 169 90
243 169 90 N 1 -> 243 170 90
243 170 90 W 3 -> 240 170 90
240 170 90 F 92 -> 240 262 90
240 262 90 N 5 -> 240 267 90
240 267 90 F 33 ->

(147, 407)